# OVR GPU Metrics

In [ ]:
import pandas as pd
import numpy as np
import os


def reset_timestamp(df):
    df['timestamp'] = df['timestamp'] - df['timestamp'][0]
    return df

def process_line(line):
    timestamp = line.split(' - ')[0]
    content = line.split(' - ')[1]
    key = str(content.split(':')[0].strip())
    value = float(content.split(':')[1].strip())
    return timestamp, key, value

def add_to_dataframe(df, timestamp, key, value):
    if key not in df.columns:
        df[key] = np.nan
    df.loc[timestamp, key] = value
    return df

def create_dataframe(file_path):
    df = pd.DataFrame()
    for line in open(file_path, 'r', encoding='utf-16'):
        if len(line) < 27:
            continue
        timestamp, key, value = process_line(line)
        df = add_to_dataframe(df, timestamp, key, value)

    df.index.name = 'timestamp'
    df = df.reset_index()
    # df = reset_timestamp(df)

    df['device'] = file_path.split('\\')[1]
    df['app'] = file_path.split('\\')[2].strip('-W')
    df['app_details'] = file_path.split('\\')[3]
    df['is_record'] = 'record' in file_path.split('\\')[4]
    df['is_wireless'] = '-W' in file_path.split('\\')[2]
    df['repetition'] = file_path.split('\\')[4].strip('record') if 'record' in file_path.split('\\')[4] else file_path.split('\\')[4].strip('replay')

    return df

def process_OVR():
    df_aggregated = pd.DataFrame()
    for root, dirs, files in os.walk(".", topdown=False):
        for name in files:
            if name == 'ovr_gpu_metrics.log':
                print(os.path.join(root, name))
                df_aggregated = pd.concat([df_aggregated, create_dataframe(os.path.join(root, name))]) 
    # df_aggregated
    return df_aggregated


In [ ]:
ovr_df = process_OVR()
ovr_df

.\Quest-2\BeatSaber\PopStars-Easy\record0\ovr_gpu_metrics.log
.\Quest-2\BeatSaber\PopStars-Easy\replay0.0\ovr_gpu_metrics.log
.\Quest-2\BeatSaber\PopStars-Easy\replay0.1\ovr_gpu_metrics.log
.\Quest-2\BeatSaber\PopStars-Easy\replay0.2\ovr_gpu_metrics.log
.\Quest-2\BeatSaber\PopStars-Hard\record0\ovr_gpu_metrics.log
.\Quest-2\BeatSaber\PopStars-Hard\replay0.0\ovr_gpu_metrics.log
.\Quest-2\BeatSaber\PopStars-Hard\replay0.1\ovr_gpu_metrics.log
.\Quest-2\BeatSaber\PopStars-Hard\replay0.2\ovr_gpu_metrics.log
.\Quest-2\BeatSaber\PopStars-Medium\record0\ovr_gpu_metrics.log
.\Quest-2\BeatSaber\PopStars-Medium\record1\ovr_gpu_metrics.log
.\Quest-2\BeatSaber\PopStars-Medium\record2\ovr_gpu_metrics.log
.\Quest-2\BeatSaber\PopStars-Medium\record3\ovr_gpu_metrics.log
.\Quest-2\BeatSaber\PopStars-Medium\record4\ovr_gpu_metrics.log
.\Quest-2\BeatSaber\PopStars-Medium\replay0.0\ovr_gpu_metrics.log
.\Quest-2\BeatSaber\PopStars-Medium\replay0.1\ovr_gpu_metrics.log
.\Quest-2\BeatSaber\PopStars-Medium\repl

,timestamp,Clocks / Second,GPU Frequency,GPU % Bus Busy,Preemptions / second,Avg Preemption Delay,% Vertex Fetch Stall,% Texture Fetch Stall,% Texture L1 Miss,% Texture L2 Miss,...,% Nearest Filtered,% Linear Filtered,% Anisotropic Filtered,% Non-Base Level Textures,device,app,app_details,is_record,is_wireless,repetition
0,12/5/2023 3:06:05 PM,266032064.0,525000000.0,33.312,168.880,26.754,1.227,32.984,30.415,7.687,...,0.002,99.986,0.002,0.018,Quest-2,BeatSaber,PopStars-Easy,True,False,0
1,12/5/2023 3:06:06 PM,270955808.0,525000000.0,31.584,203.993,15.596,1.225,31.561,30.418,7.554,...,0.002,100.000,0.002,0.001,Quest-2,BeatSaber,PopStars-Easy,True,False,0
2,12/5/2023 3:06:07 PM,267818560.0,525000000.0,29.295,199.131,11.821,1.272,31.142,30.539,7.216,...,0.002,100.000,0.002,0.001,Quest-2,BeatSaber,PopStars-Easy,True,False,0
3,12/5/2023 3:06:08 PM,212911472.0,525000000.0,49.151,69.703,2.869,1.571,51.449,31.137,7.377,...,0.001,100.000,0.001,0.000,Quest-2,BeatSaber,PopStars-Easy,True,False,0
4,12/5/2023 3:06:09 PM,263309680.0,525000000.0,29.755,199.035,8.325,1.388,31.402,30.661,7.491,...,0.002,99.986,0.002,0.016,Quest-2,BeatSaber,PopStars-Easy,True,False,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211,12/7/2023 4:30:50 PM,300047840.0,525000000.0,30.153,204.046,304.678,1.486,28.066,31.773,9.395,...,0.156,99.826,0.001,0.025,Quest-Pro,PistolWhip,BlackMagic-Easy,False,True,0.0
212,12/7/2023 4:30:51 PM,313039136.0,525000000.0,31.258,216.728,348.529,1.401,28.917,31.665,9.458,...,0.167,99.811,0.001,0.029,Quest-Pro,PistolWhip,BlackMagic-Easy,False,True,0.0
213,12/7/2023 4:30:52 PM,300441344.0,525000000.0,30.627,202.468,286.375,1.476,28.519,31.711,9.564,...,0.155,99.855,0.001,0.000,Quest-Pro,PistolWhip,BlackMagic-Easy,False,True,0.0
214,12/7/2023 4:30:53 PM,305568256.0,525000000.0,30.697,203.733,335.984,1.477,29.020,31.766,9.535,...,0.154,99.874,0.001,0.000,Quest-Pro,PistolWhip,BlackMagic-Easy,False,True,0.0


In [ ]:
ovr_df = ovr_df.reset_index()
ovr_df

,index,timestamp,Clocks / Second,GPU Frequency,GPU % Bus Busy,Preemptions / second,Avg Preemption Delay,% Vertex Fetch Stall,% Texture Fetch Stall,% Texture L1 Miss,...,% Nearest Filtered,% Linear Filtered,% Anisotropic Filtered,% Non-Base Level Textures,device,app,app_details,is_record,is_wireless,repetition
0,0,12/5/2023 3:06:05 PM,266032064.0,525000000.0,33.312,168.880,26.754,1.227,32.984,30.415,...,0.002,99.986,0.002,0.018,Quest-2,BeatSaber,PopStars-Easy,True,False,0
1,1,12/5/2023 3:06:06 PM,270955808.0,525000000.0,31.584,203.993,15.596,1.225,31.561,30.418,...,0.002,100.000,0.002,0.001,Quest-2,BeatSaber,PopStars-Easy,True,False,0
2,2,12/5/2023 3:06:07 PM,267818560.0,525000000.0,29.295,199.131,11.821,1.272,31.142,30.539,...,0.002,100.000,0.002,0.001,Quest-2,BeatSaber,PopStars-Easy,True,False,0
3,3,12/5/2023 3:06:08 PM,212911472.0,525000000.0,49.151,69.703,2.869,1.571,51.449,31.137,...,0.001,100.000,0.001,0.000,Quest-2,BeatSaber,PopStars-Easy,True,False,0
4,4,12/5/2023 3:06:09 PM,263309680.0,525000000.0,29.755,199.035,8.325,1.388,31.402,30.661,...,0.002,99.986,0.002,0.016,Quest-2,BeatSaber,PopStars-Easy,True,False,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17749,211,12/7/2023 4:30:50 PM,300047840.0,525000000.0,30.153,204.046,304.678,1.486,28.066,31.773,...,0.156,99.826,0.001,0.025,Quest-Pro,PistolWhip,BlackMagic-Easy,False,True,0.0
17750,212,12/7/2023 4:30:51 PM,313039136.0,525000000.0,31.258,216.728,348.529,1.401,28.917,31.665,...,0.167,99.811,0.001,0.029,Quest-Pro,PistolWhip,BlackMagic-Easy,False,True,0.0
17751,213,12/7/2023 4:30:52 PM,300441344.0,525000000.0,30.627,202.468,286.375,1.476,28.519,31.711,...,0.155,99.855,0.001,0.000,Quest-Pro,PistolWhip,BlackMagic-Easy,False,True,0.0
17752,214,12/7/2023 4:30:53 PM,305568256.0,525000000.0,30.697,203.733,335.984,1.477,29.020,31.766,...,0.154,99.874,0.001,0.000,Quest-Pro,PistolWhip,BlackMagic-Easy,False,True,0.0


In [ ]:
ovr_df.to_csv('ovr_overall.csv', index=False)